In [1]:
import pandas as pd     
from SmilesPE.pretokenizer import atomwise_tokenizer
import pdb      #pdb.set_trace()
import pubchempy as pcp

from IPython.display import Audio
#sound_file = '../applause.mp3'
sound_file = '../diving.mp3'

## Lenselink

In [2]:
le_s1 = pd.read_table('../datasets/Lenselink_et_al/Lenselink_sample_1.txt', usecols=['CMP_CHEMBL_ID'])
le_s2 = pd.read_table('../datasets/Lenselink_et_al/Lenselink_sample_2.txt', usecols=['CMP_CHEMBL_ID'])
le_s3 = pd.read_table('../datasets/Lenselink_et_al/Lenselink_sample_3.txt', usecols=['CMP_CHEMBL_ID'])
le_s4 = pd.read_table('../datasets/Lenselink_et_al/Lenselink_sample_4.txt', usecols=['CMP_CHEMBL_ID'])
le_s5 = pd.read_table('../datasets/Lenselink_et_al/Lenselink_sample_5.txt', usecols=['CMP_CHEMBL_ID'])

In [3]:
lense = le_s1.append(le_s2)
lense = lense.append(le_s3)
lense = lense.append(le_s4)
lense = lense.append(le_s5)

In [4]:
# backup for below
lense2 = lense

In [ ]:
chembl = pd.read_table('../datasets/ChEMBL/chembl_27_chemreps.txt', usecols=['chembl_id', 'canonical_smiles'])

In [ ]:
lense = pd.merge(lense, chembl,
                how='left',
                left_on='CMP_CHEMBL_ID', right_on='chembl_id')

In [ ]:
lense = lense[['chembl_id', 'canonical_smiles']]

In [ ]:
lense.dropna(how='any', inplace=True)
lense.reset_index(drop=True)

In [ ]:
lense['tokens'] = ''
for row in lense.index:
    lense.at[row, 'tokens'] = atomwise_tokenizer(lense.at[row, 'canonical_smiles'])

In [ ]:
lense.head()

In [ ]:
type(lense.tokens)

In [ ]:
alphabet_lense = set()

for i in lense.tokens:
    alphabet_lense.update(i)

In [ ]:
alphabet_lense

#### Alternative Mapping to get SMILES created by PubChem

In [5]:
unichem = pd.read_table('../datasets/UniChem_Mapping/src1src22.txt')
unichem.rename(columns={"From src:'1'": "chembl_id", "To src:'22'": "pubchem_id"}, inplace=True)
unichem = unichem.astype(dtype={'pubchem_id':'object'})

In [6]:
lense2 = pd.merge(lense2, unichem,
                  how='left',
                  left_on='CMP_CHEMBL_ID', right_on='chembl_id')
lense2.drop_duplicates(subset='pubchem_id', keep='first', inplace=True, ignore_index=True)
lense2.dropna(axis=0, how='any', inplace=True)
lense2.reset_index(drop=True)

,CMP_CHEMBL_ID,chembl_id,pubchem_id
0,CHEMBL3237949,CHEMBL3237949,90655448
1,CHEMBL2381895,CHEMBL2381895,68533305
2,CHEMBL2380718,CHEMBL2380718,71624117
3,CHEMBL2435858,CHEMBL2435858,58834953
4,CHEMBL2413735,CHEMBL2413735,71769385
...,...,...,...
965,CHEMBL2032165,CHEMBL2032165,57325593
966,CHEMBL2153490,CHEMBL2153490,136206394
967,CHEMBL1766798,CHEMBL1766798,52937660
968,CHEMBL1198756,CHEMBL1198756,46883362


Now I use PubChemPy to look up the canonical SMILES for that PubChemID.

In [7]:
lense2['canonical_smiles'] = ''
for row in lense2.index:
    entry = pcp.Compound.from_cid(lense2.iat[row, 2])
    lense2.iat[row, 3] = entry.canonical_smiles
    
Audio(sound_file, autoplay=True) # to notify me when it's done (this cell takes several minutes to run)

IndexError: index 970 is out of bounds for axis 0 with size 970

ATTENTION: I don't understand the error from the cell above, but the code works nonethless (I checked with the PubChem Website).

In [10]:
lense2['tokens'] = ''
for row in lense2.index:
    lense2.at[row, 'tokens'] = atomwise_tokenizer(lense2.at[row, 'canonical_smiles'])

In [13]:
alphabet_lense2 = set()

for i in lense2.tokens:
    alphabet_lense2.update(i)

In [23]:
print(alphabet_lense2)

{'Cl', '[NH3+]', ')', '3', '8', 'O', 'P', 'F', 'S', '[O]', '[O-]', 'N', '[N-]', '1', '=', 'Br', '4', '#', '6', '7', 'I', '(', '9', '5', '[N+]', 'C', '2'}


## PubChem

In [15]:
pc_s1 = pd.read_table('../datasets/PubChem/PubChem_sample_1.txt', header=None, usecols=[1])
pc_s2 = pd.read_table('../datasets/PubChem/PubChem_sample_2.txt', header=None, usecols=[1])
pc_s3 = pd.read_table('../datasets/PubChem/PubChem_sample_3.txt', header=None, usecols=[1])
pc_s4 = pd.read_table('../datasets/PubChem/PubChem_sample_4.txt', header=None, usecols=[1])
pc_s5 = pd.read_table('../datasets/PubChem/PubChem_sample_5.txt', header=None, usecols=[1])

In [16]:
pubchem = pc_s1.append(pc_s2)
pubchem = pubchem.append(pc_s3)
pubchem = pubchem.append(pc_s4)
pubchem = pubchem.append(pc_s5)

In [17]:
pubchem.dropna(how='any', inplace=True)
pubchem.reset_index(drop=True)

,1
0,C1CC(=O)N(C1=O)CCC#N
1,CC(CCC(=O)C(C)C1C(=O)CC2[C@]1(CCC3[C@@H]2CC=C4...
2,CC(C)[C@@H](C1=CC=C(C=C1)Cl)C(=O)NC2=NC=CS2
3,COC1=CC=C(C=C1)N(CC(=O)NC2=CC(=CC=C2)SC)S(=O)(...
4,CC1=CC(=CC=C1)N2C(=O)/C(=C\C3=CC=C(C=C3)O[C@H]...
...,...
995,C1C(N(C2=CC=CC(=C2O1)C3=CC(=CC=C3)OC(F)(F)F)C[...
996,CCC=NC1=CC=CC=C1SC2=CC=CC=C2.CC(/C=C(\C(=C/CC=...
997,C/C=C\C=C(\C1CCN(CC1)C=O)/C(=C\C)/C(F)(F)F
998,CC(C)(O)O.C=CC1CCCC2=C(C3=C(C(=C12)C4=CC=CC=C4...


In [18]:
pubchem.rename(columns={1:'smiles'}, inplace=True)

In [19]:
alphabet_pubchem = set()

for row in pubchem.index:
    tokens = atomwise_tokenizer(pubchem.iloc[row,0])
    alphabet_pubchem.update(tokens)

In [20]:
alphabet_pubchem

{'#',
 '(',
 ')',
 '.',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '=',
 'Br',
 'C',
 'Cl',
 'F',
 'I',
 'N',
 'O',
 'P',
 'S',
 '[B]',
 '[Br-]',
 '[C-]',
 '[C@@H]',
 '[C@@]',
 '[C@H]',
 '[C@]',
 '[CH2]',
 '[HH]',
 '[I-]',
 '[Mg+2]',
 '[N+]',
 '[N-]',
 '[NH+]',
 '[NH2+]',
 '[Ni+2]',
 '[O-]',
 '[Sn]',
 '[Te]',
 '\\'}

## Compare the alphabets

In [21]:
alphabet_lense2.difference(alphabet_pubchem) # die Elemente, die nicht in PubChem sind

{'[NH3+]', '[O]'}

In [22]:
alphabet_pubchem.difference(alphabet_lense2) # die Elemente, die nicht in Lense sind

{'.',
 '/',
 '[B]',
 '[Br-]',
 '[C-]',
 '[C@@H]',
 '[C@@]',
 '[C@H]',
 '[C@]',
 '[CH2]',
 '[HH]',
 '[I-]',
 '[Mg+2]',
 '[NH+]',
 '[NH2+]',
 '[Ni+2]',
 '[Sn]',
 '[Te]',
 '\\'}

In [ ]:
#alphabet_lense.intersection(alphabet_pubchem)

Strategy: Throw out the things in PubChem, which contain tokens which aren't in Lense.

In [ ]:
print(alphabet_lense)

In [24]:
shared_alphabet = alphabet_pubchem & alphabet_lense2

In [26]:
print(shared_alphabet)

{'Cl', ')', '3', '8', 'O', 'P', 'F', 'S', '[O-]', 'N', '[N-]', '1', '=', 'Br', '4', '#', '6', '7', 'I', '(', '9', '5', '[N+]', 'C', '2'}
